# Update Production Data and Model

> author: Shizhenkun   
> email: zhenkun.shi@tib.cas.cn   
> date: 2021-12-24  

This file contains update codes for the production server. The update should be scheduled every eight weeks.

## 1. Import packages

In [115]:
import numpy as np
import pandas as pd

import os
import re
import time
import datetime
import sys
from tqdm import tqdm
sys.path.append("../tools/")
import funclib

import config as cfg
from functools import reduce
import matplotlib.pyplot as plt

sys.path.append("./tools/")
import exact_ec_from_uniprot as exactec
import minitools as mtool
import embedding_esm as esmebd
# import embedding_unirep as unirep

from pandarallel import pandarallel 
pandarallel.initialize() 
import benchmark_train as btrain

%load_ext autoreload
%autoreload 2

INFO: Pandarallel will run on 32 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.
The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## 2. Define Functions

In [3]:
# install axel for download dataset
def install_axel():
    isExists = !which axel
    if 'axel' in str(isExists[0]):
        return True
    else:
        !sudo apt install axel -y

# add missing '-' for ec number
def refill_ec(ec):   
    if ec == '-':
        return ec
    levelArray = ec.split('.')
    if  levelArray[3]=='':
        levelArray[3] ='-'
    ec = '.'.join(levelArray)
    return ec

def specific_ecs(ecstr):
    if '-' not in ecstr or len(ecstr)<4:
        return ecstr
    ecs = ecstr.split(',')
    if len(ecs)==1:
        return ecstr
    
    reslist=[]
    
    for ec in ecs:
        recs = ecs.copy()
        recs.remove(ec)
        ecarray = np.array([x.split('.') for x in recs])
        
        if '-' not in ec:
            reslist +=[ec]
            continue
        linearray= ec.split('.')
        if linearray[1] == '-':
            #l1 in l1s and l2 not empty
            if (linearray[0] in  ecarray[:,0]) and (len(set(ecarray[:,0]) - set({'-'}))>0):
                continue
        if linearray[2] == '-':
            # l1, l2 in l1s l2s, l3 not empty
            if (linearray[0] in  ecarray[:,0]) and (linearray[1] in  ecarray[:,1]) and (len(set(ecarray[:,2]) - set({'-'}))>0):
                continue
        if linearray[3] == '-':
            # l1, l2, l3 in l1s l2s l3s, l4 not empty
            if (linearray[0] in  ecarray[:,0]) and (linearray[1] in  ecarray[:,1]) and (linearray[2] in  ecarray[:,2]) and (len(set(ecarray[:,3]) - set({'-'}))>0):
                continue
                
        reslist +=[ec]
    return ','.join(reslist)

#format ec
def format_ec(ecstr):
    ecArray= ecstr.split(',')
    ecArray=[x.strip() for x in ecArray] #strip blank
    ecArray=[refill_ec(x) for x in ecArray] #format ec to full
    ecArray = list(set(ecArray)) # remove duplicates
    
    return ','.join(ecArray)

## 3. Download latest data from unisprot

In [ ]:
# download location ./tmp

! mv $cfg.DATADIR'uniprot_sprot_latest.dat.gz' $cfg.TEMPDIR$currenttime'_uniprot_sprot_latest.dat.gz'
install_axel()
!axel -n 10 https://ftp.uniprot.org/pub/databases/uniprot/current_release/knowledgebase/complete/uniprot_sprot.dat.gz -o ./data/uniprot_sprot_latest.dat.gz -q -c

## 4. Preprocessing

In [21]:
exactec.run_exact_task(infile=cfg.DATADIR+'uniprot_sprot_latest.dat.gz', outfile=cfg.DATADIR+'sprot_latest.tsv')

#加载数据并转换时间格式
sprot_latest = pd.read_csv(cfg.DATADIR+'sprot_latest.tsv', sep='\t',header=0) #读入文件
sprot_latest = mtool.convert_DF_dateTime(inputdf = sprot_latest)

sprot_latest.drop_duplicates(subset=['seq'], keep='first', inplace=True)
sprot_latest.reset_index(drop=True, inplace=True)

#sprot_latest format EC
sprot_latest['ec_number'] = sprot_latest.ec_number.parallel_apply(lambda x: format_ec(x))
sprot_latest['ec_number'] = sprot_latest.ec_number.parallel_apply(lambda x: specific_ecs(x))
sprot_latest['functionCounts'] = sprot_latest.ec_number.parallel_apply(lambda x: 0 if x=='-'  else len(x.split(',')))

#get trainset
train = sprot_latest.iloc[:,np.r_[0,2:7,10:12]]

# Trim Strging
with pd.option_context('mode.chained_assignment', None):
    train.ec_number = train.ec_number.parallel_apply(lambda x : str(x).strip()) #ec trim
    train.seq = train.seq.parallel_apply(lambda x : str(x).strip()) #seq trim
# Save train    
train.to_feather(cfg.DATADIR + 'latest_sprot.feather')

/home/shizhenkun/codebase/DMLF/data/uniprot_sprot_latest.dat.gz /home/shizhenkun/codebase/DMLF/tmp/20211224_1130_uniprot_sprot_latest.dat.gz


## 5. Caculation Features

In [24]:
train= pd.read_feather(cfg.DATADIR + 'latest_sprot.feather')
print('train size: {0}'.format(len(train)))

train size: 477917


In [ ]:
! mv $cfg.DATADIR'sprot_latest_rep0.feather' $cfg.DATADIR'featureBank/sprot_latest_rep0.feather'
! mv $cfg.DATADIR'sprot_latest_rep32.feather' $cfg.DATADIR'featureBank/sprot_latest_rep32.feather'
! mv $cfg.DATADIR'sprot_latest_rep33.feather' $cfg.DATADIR'featureBank/sprot_latest_rep33.feather'
! mv $cfg.DATADIR'sprot_latest_unirep.feather' $cfg.DATADIR'featureBank/sprot_latest_unirep.feather'

# !pip install fair-esm
tr_rep0, tr_rep32, tr_rep33 = esmebd.get_rep_multi_sequence(sequences=train, model='esm1b_t33_650M_UR50S',seqthres=1022)
tr_rep0.to_feather(cfg.DATADIR + 'sprot_latest_rep0.feather')
tr_rep32.to_feather(cfg.DATADIR + 'sprot_latest_rep32.feather')
tr_rep33.to_feather(cfg.DATADIR + 'sprot_latest_rep33.feather')



In [32]:
train_esm_latest = pd.read_feather(cfg.DATADIR + 'sprot_latest_rep32.feather')
train_esm_latest = train.merge(train_esm_latest, on='id', how='left')

## 6. Split X Y

In [123]:
# task 1
X_train_task1 = np.array(train_esm_latest.iloc[:,8:])
Y_train_task1 = np.array(train_esm_latest.isenzyme.astype('int')).flatten()
train_enzyme = train_esm_latest[train_esm_latest.isenzyme].reset_index(drop=True)

# task 2
X_train_task2_s = np.array(train_enzyme.iloc[:,8:])
Y_train_task2_s = train_enzyme.functionCounts.apply(lambda x : 0 if x==1 else 1).astype('int').values

train_task2M=train_enzyme[train_enzyme.functionCounts>=2].reset_index(drop=True)
X_train_task2_m = np.array(train_task2M.iloc[:,8:])
Y_train_task2_m = np.array(train_task2M.functionCounts.astype('int')-2).flatten()

#task 3
train_set_task3= funclib.split_ecdf_to_single_lines(train_enzyme.iloc[:,np.r_[0,6,4]])
train_set_task3=train_set_task3.merge(train_esm_latest.iloc[:,np.r_[0,8:1288]], on='id', how='left')

#4. 加载EC号训练数据
print('loading ec to label dict')
dict_ec_label = btrain.make_ec_label(train_label=train_set_task3['ec_number'], test_label=train_set_task3['ec_number'], file_save= cfg.FILE_EC_LABEL_DICT, force_model_update=cfg.UPDATE_MODEL)

train_set_task3['ec_label']=train_set_task3.ec_number.parallel_apply(lambda x: dict_ec_label.get(x))    
X_train_task3 = np.array(train_set_task3.iloc[:,3:])
Y_train_task3 = np.array(train_set_task3.ec_label.astype('int')).flatten()

100%|█████████████████████████████████████████████████████████████████████| 231373/231373 [05:17<00:00, 729.21it/s]


loading ec to label dict


## 7. Train Model

In [ ]:
# Task 1
groundtruth, predict, predictprob, model = funclib.knnmain(X_train_task1, Y_train_task1, X_train_task1.iloc[0:10,:], Y_train_task1.iloc[0:10,:], type='binary')
joblib.dump(model, cfg.ISENZYME_MODEL)

In [ ]:
groundtruth, predict, predictprob, model = funclib.xgmain(X_train_task2_s, Y_train_task2_s, X_train_task2_s.iloc[0:10,:], Y_train_task2_s.iloc[0:10,:], type='binary')
joblib.dump(model, cfg.MODELDIR+'/single_multi.model')

groundtruth, predict, predictprob, model = funclib.xgmain(X_train_task2_m, Y_train_task2_m, X_train_task2_m.iloc[0:10,:], Y_train_task2_m.iloc[0:10,:], type='multi')
joblib.dump(model, cfg.MODELDIR+'/multi_many.model')

In [ ]:
cfg.FEATURE_NUM = 1280
#train
bcommon.prepare_slice_file(x_data=X_train_task3, y_data=Y_train_task3, x_file=cfg.DATADIR+'slice_train_x_esm32_latest.txt', y_file=cfg.DATADIR+'slice_train_y_esm32_latest.txt', ec_label_dict=dict_ec_label)
btrain.train_ec_slice(trainX=cfg.DATADIR+'slice_train_x_esm32_latest.txt', trainY=cfg.DATADIR+'slice_train_y_esm32_latest.txt', modelPath=cfg.MODELDIR+'/slice_esm32')



In [129]:
latest_sprot = pd.read_feather(cfg.FILE_LATEST_SPROT_FEATHER)

In [130]:
latest_sprot

,id,isenzyme,isMultiFunctional,functionCounts,ec_number,ec_specific_level,seq,seqlength
0,P84233,False,False,0,-,0,MARTKQTARKSTGGKAPRKQLATKAARKSAPATGGVKKPHRYRPGT...,136
1,P0A7F3,False,False,0,-,0,MTHDNKLQVEAIKRGTVIDHIPAQIGFKLLSLFKLTETDQRITIGL...,153
2,P03212,False,False,0,-,0,MRAVGVFLAICLVTIFVLPTWGNWAYPCCHVTQLRAQHLLALENIS...,137
3,P01158,False,False,0,-,0,WAGGDASGE,9
4,P0A6U4,False,False,0,-,0,MFYPDPFDVIIIGGGHAGTEAAMAAARMGQQTLLLTHNIDTLGQMS...,629
...,...,...,...,...,...,...,...,...
477912,A0A0U5GMR5,False,False,0,-,0,MTTTRHRLLATASRFVTTLESLDVDAMLAVRSPTCLHHMCLPSFRN...,165
477913,S8FIE4,True,False,1,1.10.3.2,4,MAFTAISLFLAALGVINTAFAQSAVIGPVTDLDIINAEVNLDGFPR...,539
477914,S8FGV1,True,False,1,1.10.3.2,4,MLLSSAFVGSCLAILNFAAAVSAQGGLSRTTLNIVNKVISPDGYSR...,530
477915,Q57286,True,False,1,7.2.4.3,4,MEAFAVAIQSVINDSGFLAFTTGNAIMILVGLILLYLAFAREFEPL...,373


In [136]:
input_df = funclib.load_fasta_to_table('/home/shizhenkun/codebase/DMLF/data/test1.fasta')

In [137]:
input_df

,id,seq
0,t1,MAFLKRRDLISIKSLFLVLFLGLVSLSICEQEKREEENQEEDEENE...
1,t2,MAFLKKSLFLVLFLGKKKKKKKKKKKLVSLSICEKEKRQNEEDEDE...
2,t3,MARGGDTGCTGPSETSASMMMMMMMMMGAVAIAFPGLEGPPADAQY...
3,t4,KERKTGMFVMHEIGTPMEVRESGCDDKHHPHLKMMRYDEFADAILW...
4,r1,HLYCMFAQQATADYGGGNQYSYPYMPPGCWLPWLYNQCKQDHDDCN...
...,...,...
459,F4KH86,MDSDAWEIIHIPEKPSLSPDHQPTVKVYASLIKPRFANTIVRHLCK...
460,Q81LM1,MISVNKVFYAHSERFQMQNMNVHIKAGEVVSLIGPNGSGKSTLLRL...
461,Q9LS80,MVRNQSRCFLGIIITISFFFFLLSLPNTFASPTRSLCRSDQRDALL...
462,I1RW37,MHQQPRGPARVSSPGATTQPNLPSRSNSTREAALGSRPRAGSNLVG...


In [138]:
find_data =input_df.merge(latest_sprot, on='seq', how='left')

In [139]:
find_data

,id_x,seq,id_y,isenzyme,isMultiFunctional,functionCounts,ec_number,ec_specific_level,seqlength
0,t1,MAFLKRRDLISIKSLFLVLFLGLVSLSICEQEKREEENQEEDEENE...,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,t2,MAFLKKSLFLVLFLGKKKKKKKKKKKLVSLSICEKEKRQNEEDEDE...,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,t3,MARGGDTGCTGPSETSASMMMMMMMMMGAVAIAFPGLEGPPADAQY...,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,t4,KERKTGMFVMHEIGTPMEVRESGCDDKHHPHLKMMRYDEFADAILW...,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,r1,HLYCMFAQQATADYGGGNQYSYPYMPPGCWLPWLYNQCKQDHDDCN...,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...
459,F4KH86,MDSDAWEIIHIPEKPSLSPDHQPTVKVYASLIKPRFANTIVRHLCK...,F4KH86,True,False,1.0,3.5.4.33,4.0,400.0
460,Q81LM1,MISVNKVFYAHSERFQMQNMNVHIKAGEVVSLIGPNGSGKSTLLRL...,Q81LM1,True,False,1.0,7.2.2.-,3.0,272.0
461,Q9LS80,MVRNQSRCFLGIIITISFFFFLLSLPNTFASPTRSLCRSDQRDALL...,Q9LS80,False,False,0.0,-,0.0,835.0
462,I1RW37,MHQQPRGPARVSSPGATTQPNLPSRSNSTREAALGSRPRAGSNLVG...,I1RW37,False,False,0.0,-,0.0,927.0


In [144]:
find_data[~find_data.id_y.isnull()].rename(columns={'id_x':'id','id_y':'id_uniprot'})

,id,seq,id_uniprot,isenzyme,isMultiFunctional,functionCounts,ec_number,ec_specific_level,seqlength
14,P69031,MAFLKKSLFLVLFLGLVSLSICEQEKREEENQEEDEENEAASEEKR...,P69031,False,False,0.0,-,0.0,71.0
15,P69019,MAFLKKSLFLVLFLGLVSLSICEKEKRQNEEDEDENEAANHEEGSE...,P69019,False,False,0.0,-,0.0,72.0
16,Q5RF96,MARGGDTGCTGPSETSASGAVAIAFPGLEGPPADAQYQTLALTVPK...,Q5RF96,True,False,1.0,3.4.-.-,2.0,169.0
17,Q9UM00,MPRKRKCDLRAVRVGLLLGGGGVYGSRFRFTFPGCRALSPWRVRVQ...,Q9UM00,False,False,0.0,-,0.0,239.0
18,P0C250,MGKLTILVLVAAVLLSTQVMVQGDGDQPADRNAVRRDDNPGGTRGR...,P0C250,False,False,0.0,-,0.0,63.0
...,...,...,...,...,...,...,...,...,...
459,F4KH86,MDSDAWEIIHIPEKPSLSPDHQPTVKVYASLIKPRFANTIVRHLCK...,F4KH86,True,False,1.0,3.5.4.33,4.0,400.0
460,Q81LM1,MISVNKVFYAHSERFQMQNMNVHIKAGEVVSLIGPNGSGKSTLLRL...,Q81LM1,True,False,1.0,7.2.2.-,3.0,272.0
461,Q9LS80,MVRNQSRCFLGIIITISFFFFLLSLPNTFASPTRSLCRSDQRDALL...,Q9LS80,False,False,0.0,-,0.0,835.0
462,I1RW37,MHQQPRGPARVSSPGATTQPNLPSRSNSTREAALGSRPRAGSNLVG...,I1RW37,False,False,0.0,-,0.0,927.0
